# B) Linear system from optimality condition of MSE

- **<font color='green'>[RUN & OBSERVE]</font>** $\rightarrow$ the cell should be run directly without modification
- **<font color='orange'>[RUN & PLAY]</font>** $\rightarrow$ the cell can be run directly, but some parameters should be changed interactively
- **<font color='red'>[FILL & RUN]</font>**    $\rightarrow$ the cell should be filled before being run
- **<font color='magenta'>[FILL & PLAY]</font>** $\rightarrow$ the cell should be filled, and then some parameters should be changed interactively.

_______
## 1) Reference function

We consider the following analytical function supported on a unit square :
$$ u_0 : \left \{ \begin{array}{rcl}
\Omega = [0,1]^2 & \rightarrow & \mathbb R \\
(x_1 , x_2) &\mapsto & (x_1-0.5)^2 + (x_2-0.5)^2
\end{array} \right.$$

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from utils.myGeometries import square
from ngsolve.webgui import Draw
from ngsolve import x, y
Omega = square(maxh=0.2)          # generates the domain and its disretization
u0 = (x-0.5)**2 + (y-0.5)**2   # define the function
Draw(u0, Omega)

_________
## 2) Variational formulation
### a) Stationarity 
Actually, one can solve the least-square problem in one step. Since it is a **convex unconstrained minimization problem**, it is enough to write the **optimality condition**, i.e, the derivative cancellation of $J$ :

$$ \forall v\in H(\Omega), \quad  J'(u;v) = 0$$
$$ \Rightarrow \forall v\in H(\Omega),  \quad \int_\Omega (u-u_0) v = 0 $$
$$ \Rightarrow \forall v\in H(\Omega),  \quad \underbrace{\int_\Omega u v}_{A(u,v)} = \underbrace{\int_\Omega u_0 v}_{l(v)} $$

- The arbitrary direction $v$ is called a **test function**, from the chosen function space $H(\Omega)$
- The left-hand side $A(u,v)$ is a bilinear form
- The right hand side $l(v)$ is a linear form

### b) Weak form

- The last line is called a **variational formulation**; it represents the **weak form** of the following "**strong**" equation, obtained by removing the integrals and test functions :
$$ u = u_0  $$
- The variational form is called **weak** because $u$ might not be exactly equal to $u_0$, it is only the *best possible* in the admissible space $H(\Omega)$.
- $u_0$ is always solution of the weak form, but the converse is not be true.

`NGSolve` is a toolbox to solve variational formulations numerically. We can define linear and bilinear form, with the syntaxes :
- `A = BilinearForm( <symbolic expression> * dx)`
- or `A = BilinearForm(fes)`, and then `A += <symbolic expression> * dx` for long expressions.
- `A += <symbolic expression> * dx("regionLabel")` is equivalent to restric the integral over a specific region.
|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
#-------------------------------------------
# Finite element space
from ngsolve import H1, L2
order = 0
fes = L2(Omega, order = order)
#fes = H1(Omega, order = order)
#-------------------------------------------
# Definition of linear and bilinear forms
from ngsolve import dx, BilinearForm, LinearForm
u = fes.TrialFunction()
v = fes.TestFunction()
A = BilinearForm( u*v * dx )  # define the bilinear form
l = LinearForm( u0*v * dx )   # define the linear form

______
## 2) Discretization and assembly

Once discretized on a mesh and suitable function space with DoF indexed by $i$, we formally have:

- Bilinear form is equivalent to a matrix : $K_{ij} = A(u_j, v_i) $
- Linear form is equivalent to a vector $f_i = l(v_i)$

with $u_i, v_j$ the unitary function associated to DoF $i,j$, repsectively. Then, after discretization, one has to solve the matrix system :

$$ K_{ij} u_{j} = f_{i} $$

We'll see in the next class how the assembly process works in more details (not so simple!). What we have to remember from now is that this process is about computing elementary integrals.
Right now, we just use the capabilities of `NGSolve`.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
#Assembly step (difficult step, but NGsolve takes care of it!)
K = A.Assemble().mat
f = l.Assemble().vec

We can access the matrix and vizualize it.


### Exercise
1. Is the matrix **symmetric**? Why is it so **sparse**?
2. Observe the **sparsity patterns** of the assembled bilinear form with continuous and discontinuous function spaces (`H1` and `L2`). Why is there a difference in the sparsity pattern?

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
import scipy.sparse as sp
import matplotlib.pylab as plt
rows, cols, vals = K.COO()
print(f"Non-zero terms ratio: {len(vals)/(K.height*K.width)*100:.2f} %")
plt.spy(sp.csr_matrix((vals,(rows,cols))) , markersize = 1)
plt.show()    # show non-zero terms of the matrix

______
## 3) Solving the problem

Solving the mean-square problem is now equivalent to solving this unique matrix system, without the need to iterate.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from ngsolve import GridFunction
sol = GridFunction(fes)
sol.vec.data = K.Inverse() * f
Draw(sol,  settings = {"deformation" :  0.5})

**NB** : Actually, the inverse of the matrix is not really computed. The matrix is decomposed and put in a form (e.g., triangular) such that a linear system is now easy to solve. This is the principle of a direct solver.